In [1]:
import numpy as np

from os import listdir
from os.path import isfile, join

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 268966481426530976
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3291938816
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11967007857422258219
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [6]:
mode = 0
directory = "DATASET/mode_"+str(mode)+"/refined/"

In [7]:
X_train_raw = np.load(directory+"X_train.npy")
y_train_raw = np.load(directory+"y_train.npy")

X_val_raw = np.load(directory+"X_val.npy")
y_val_raw = np.load(directory+"y_val.npy")

X_test_raw = np.load(directory+"X_test.npy")
y_test_raw = np.load(directory+"y_test.npy")

In [12]:
print(X_train_raw.shape)
img_height = X_train_raw.shape[1]
img_width = X_train_raw.shape[2]

n_classes = 2

(1800, 320, 480)


In [13]:
n_train = y_train_raw.shape[0]
n_val = y_val_raw.shape[0]
n_test = y_test_raw.shape[0]

X_train = X_train_raw.reshape((n_train, img_height, img_width, 1))
X_val = X_val_raw.reshape((n_val, img_height, img_width, 1))
X_test = X_test_raw.reshape((n_test, img_height, img_width, 1))

# ONE HOT ENCODING
y_train = np.eye(n_classes)[y_train_raw]
y_val = np.eye(n_classes)[y_val_raw]
y_test = np.eye(n_classes)[y_test_raw]

In [23]:
print("y_train shape: ", y_train.shape)
print("Number of classes: ", n_classes)
print("y_train: \n", y_train)

y_train shape:  (1800, 2)
Number of classes:  2
y_train: 

 [[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [31]:
batch_size = 1
epochs = 12

input_shape = (img_height, img_width, 1)

print("Height: ", img_height)
print("Width: ", img_width)
print("Input shape: ", input_shape)
print("X_train shape: ", X_train.shape)
print("X_val shape: ", X_val.shape)
print("X_test shape: ", X_test.shape)

Height:  320
Width:  480
Input shape:  (320, 480, 1)
X_train shape:  (1800, 320, 480, 1)
X_val shape:  (450, 320, 480, 1)
X_test shape:  (750, 320, 480, 1)


In [33]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(5, 5),
                 activation='relu',
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Conv2D(32, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, y_val))

Train on 1800 samples, validate on 450 samples
Epoch 1/12
1800/1800 [==============================] - 24s 13ms/step - loss: 0.3510 - acc: 0.8572 - val_loss: 0.2686 - val_acc: 0.8822
Epoch 2/12
1800/1800 [==============================] - 19s 11ms/step - loss: 0.2635 - acc: 0.9022 - val_loss: 0.3637 - val_acc: 0.8733
Epoch 3/12
1800/1800 [==============================] - 19s 11ms/step - loss: 0.2490 - acc: 0.9083 - val_loss: 0.2824 - val_acc: 0.8889
Epoch 4/12
1800/1800 [==============================] - 18s 10ms/step - loss: 0.2294 - acc: 0.9094 - val_loss: 0.2928 - val_acc: 0.8933
Epoch 5/12
1800/1800 [==============================] - 18s 10ms/step - loss: 0.2189 - acc: 0.9261 - val_loss: 0.3291 - val_acc: 0.8689
Epoch 6/12
1800/1800 [==============================] - 19s 10ms/step - loss: 0.2095 - acc: 0.9294 - val_loss: 0.2703 - val_acc: 0.8889
Epoch 7/12
1800/1800 [==============================] - 19s 10ms/step - loss: 0.1980 - acc: 0.9311 - val_loss: 0.3714 - val_acc: 0.8800
E

ResourceExhaustedError: OOM when allocating tensor with shape[32,16,316,476] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: conv2d_11/convolution = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_11/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_11/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [34]:
score = model.evaluate(X_val, y_val, verbose=1, batch_size=1)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

450/450 [==============================] - 2s 4ms/step
Validation loss: 0.5989446223141547
Validation accuracy: 0.88
